In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

In [24]:
# Load datasets
add=r"C:\Users\matin\Documents\minor project[1]\minor project\INFY Historical Data (2).csv"
infosys_data = pd.read_csv(add, date_parser=True)
print(infosys_data.columns)


Index(['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'], dtype='object')


In [25]:
infosys_data.rename(columns={'Price': 'Close'}, inplace=True)
def clean_numeric_columns(df, columns):
    for col in columns:
        df[col] = df[col].astype(str).str.replace(',', '').astype(float)
    return df

# Data Conversion and Normalization

In [26]:
# Apply to both datasets
infosys_data = clean_numeric_columns(infosys_data, ['Close', 'Open', 'High', 'Low'])
infosys_data['Date'] = pd.to_datetime(infosys_data['Date'],dayfirst=True)
infosys_data.set_index('Date', inplace=True)
infosys_data=infosys_data.sort_index()
scaler = MinMaxScaler(feature_range=(0, 1))
infosys_scaled = scaler.fit_transform(infosys_data[['Close']])
def create_sequences(data, time_step=60):
    X, y = [], []
    for i in range(time_step, len(data)):
        #print(i)
        #print(data[i-time_step:i, 0])
        X.append(data[i-time_step:i, 0])
        y.append(data[i, 0])
        #print(data[i,0])
    return np.array(X), np.array(y)



# Reshape of Dataset

In [27]:
X_infy, y_infy = create_sequences(infosys_scaled,3)
x=X_infy.reshape((X_infy.shape[0],X_infy.shape[1],1))
y=y_infy
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=False,shuffle=False)
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import KFold

# Model Building

In [28]:
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(LSTM(units = 50, activation = 'relu', return_sequences=True
              ,input_shape = (3, 1)))
model.add(Dropout(0.2))


model.add(LSTM(units = 60, activation = 'relu', return_sequences=True))
model.add(Dropout(0.3))


model.add(LSTM(units = 80, activation = 'relu', return_sequences=True))
model.add(Dropout(0.4))


model.add(LSTM(units = 120, activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(units = 1))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_4 (LSTM)                        │ (None, 3, 50)               │          10,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 3, 50)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_5 (LSTM)                        │ (None, 3, 60)               │          26,640 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 3, 60)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_6 (LSTM)                        │ (None, 3, 80)               │          45,120 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 3, 80)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_7 (LSTM)                        │ (None, 120)                 │          96,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 120)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             121 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 178,761 (698.29 KB)

 Trainable params: 178,761 (698.29 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics=[tf.keras.metrics.MeanAbsoluteError()])
model.fit(x_train, y_train,epochs = 100)

Epoch 1/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - loss: 0.1679 - mean_absolute_error: 0.3468
Epoch 2/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0477 - mean_absolute_error: 0.1830
Epoch 3/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 0.0288 - mean_absolute_error: 0.1373
Epoch 4/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0111 - mean_absolute_error: 0.0814
Epoch 5/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.0087 - mean_absolute_error: 0.0701
Epoch 6/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0080 - mean_absolute_error: 0.0662
Epoch 7/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0076 - mean_absolute_error: 0.0657
Epoch 8/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 0.0082 - mean_absolute_error: 0.0668
Epoch 9/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 0.0094 - mean_absolute_error: 0.0708
Epoch 10/100
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - loss: 0.0060 - mean_absolute_error: 0.0594
Epoch 11/100
28/28 ━━━━━━━━━

In [ ]:
def pred(test,model=model,incerse=True):
    #test=test.reshape(len(test),1)
    prediction=model.predict(test)
    if incerse:
        return scaler.inverse_transform(prediction).reshape(-1)
    return prediction.reshape(-1)
pres=pred(x,model=model,incerse=True)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
y_t=pred(x_test,model,False)

In [ ]:
(r2_score(y_test,y_t))

In [ ]:
res=pd.DataFrame(data={"Date":infosys_data.index[3:],"Price":infosys_data["Close"].values[3:],"pred":pres})

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(res['Date'],res['Price'],label='Actuall Price in ₹')
plt.plot(res['Date'],res['pred'],label='Predicted Price in ₹')
plt.title("Price Graph")
plt.xlabel('Date')
plt.ylabel('Price ₹')
plt.legend()
plt.show()

In [ ]:
def reshape(x):
    x=x.reshape(1,len(x),1)
    return x


In [ ]:
input_=res.loc[len(res)-3:,['Date','Price']].reset_index(drop=True)

In [ ]:
def norm(test,n=3,scaler=scaler):
    test=np.array(test).reshape(n,1)
    return scaler.transform(test)

In [ ]:
def Output_Gen(input_,n,step=3):
    last=input_.iloc[-1,0]+timedelta(1)
    input=input_.copy()
    print(last)
    for j,i in enumerate(range(n)):
        tes=reshape(norm(input.iloc[i:i+step,1].values))
        print(tes)
        Y=pred(tes)[0]
        print(Y)
        input.loc[j+3]=[last,Y]
        print(last)
        last=last+timedelta(1)
    return input
    

In [ ]:
output=Output_Gen(input_,7)

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(res['Date'],res['Price'],label='Actuall Price in ₹')
plt.plot(res['Date'],res['pred'],label='Predicted Price in ₹')
plt.plot(output.loc[3:,'Date'],output.loc[3:,'Price'],label='Future Price in ₹')
plt.title("Price Graph")
plt.xlabel('Date')
plt.ylabel('Price ₹')
plt.legend()
plt.grid(True)
plt.show()